In [ ]:
%%html
<style>
.container {
  width: 100%;
}
</style>

In [ ]:
# Activate mypy type checking
%load_ext nb_mypy

In [ ]:
import nbimporter # Activate import functionality for notebooks

# Chess-AI Statistik

Dieses Notebook lässt verschiedene AI-Versionen gegeneinander spielen und fasst die Ergebnisse in einer CSV-Datei zusammen.

Alle Optionen für die Ausführung können über die Umgebungsvariablen gesetzt werden. Das Notebook kann auch unbeaufsichtigt über [GitHub Actions](https://github.com/piuswalter/Chess-AI/actions/workflows/statistics.yml) ausgeführt werden. Der Workflow kann dort auf jedem existierenden Branch manuell gestartet werden. Der gewählte Branch entscheidet über den verwendeten Code. Ein Durchlauf mit den Standardoptionen (`development`-Branch) dauert derzeit etwa 1 Stunde. Die maximal erlaubte Laufzeit mit GitHub Actions beträgt 6 Stunden.

Verfügbare Versionen:
- `AI-Base-Class.ipynb`: Die abstrakte Basisklasse für alle AI-Versionen, welche das Eröffnungs- und Endspiel implementiert.
- `Exercise01AI`: Die erste Version der AI, welche im Mittelspiel zufällige Züge auswählt.
- `Exercise02AI`: Die zweite Version der AI, welche im Mittelspiel Züge mithilfe des Minimax-Algorithmus und dem einfachen Materialwert auswählt.
- `Exercise03AI`: Die dritte Version der AI, welche im Mittelspiel Züge mithilfe des Minimax-Algorithmus und der `simple evaluation function` auswählt.
- `Exercise04AI`: Die vierte Version der AI, welche im Vergleich zur dritten Version mithilfe von Memoisierung beschleunigt wurde.
- `Exercise05AI`: HIER FEHLT EINE BESCHREIBUNG
- `Exercise06AI`: HIER FEHLT EINE BESCHREIBUNG
- `Exercise07AI`: HIER FEHLT EINE BESCHREIBUNG
- `StockfishPlayer`: Ein Spieler, welcher durch die Stockfish-Engine gesteuert wird.

## Optionen manuell anpassen

Die folgende Zelle kann dazu genutzt werden um die Optionen manuell anzupassen.

In [ ]:
import json
manual_config = json.loads(
'''
{
  "players": {
    "Exercise01AI": {},
    "Exercise02AI": {
      "search_depth": 3
    },
    "Exercise03AI": {
      "search_depth": 3
    },
    "Exercise04AI": {
      "search_depth": 3
    },
    "Exercise05AI": {
      "search_depth": 4
    },
    "Exercise06AI": {
      "search_depth": 4
    },
    "Exercise07AI": {
      "search_depth": 5
    },
    "StockfishPlayer": {}
  },
  "opponents": {
    "Exercise01AI": {},
    "Exercise02AI": {
      "search_depth": 3
    },
    "Exercise03AI": {
      "search_depth": 3
    },
    "Exercise04AI": {
      "search_depth": 3
    },
    "Exercise05AI": {
      "search_depth": 4
    },
    "Exercise06AI": {
      "search_depth": 4
    },
    "Exercise07AI": {
      "search_depth": 5
    },
    "StockfishPlayer": {}
  },
  "seed": 1,
  "repetitions": 1,
  "exclude_reflexive_pairs": false,
  "make_symmetric_pairs": true
}
''')

In [ ]:
import os
# Import all versions
from AIBaseClass import ChessAI
from Exercise01AI import Exercise01AI
from Exercise02AI import Exercise02AI
from Exercise03AI import Exercise03AI
from Exercise04AI import Exercise04AI
from Exercise05AI import Exercise05AI
from Exercise06AI import Exercise06AI
from Exercise07AI import Exercise07AI
from StockfishPlayer import StockfishPlayer
from Main import run_games

available_versions = [
    Exercise01AI,
    Exercise02AI,
    Exercise03AI,
    Exercise04AI,
    Exercise05AI,
    Exercise06AI,
    Exercise07AI,
    StockfishPlayer
]

Die Methode `get_config` lädt eine gegebene Konfiguration aus den Umgebungsvariablen und gibt diese zurück.

In [ ]:
def get_config() -> dict:
    env_config = os.environ.get("config")
    if env_config:
        return json.loads(env_config)
    return manual_config

Die `cleanup_game_folder`-Funktion löscht alle Dateien im `games`-Ordner. Dies verhindert, dass bereits existierende Dateien ungewollt in den Ergebnissen enthalten sind.

In [ ]:
import os

def cleanup_game_folder() -> None:
    """Deletes all files inside the games folder."""
    for filename in os.listdir("../games"):
        if filename.endswith(".pgn") or filename.endswith(".csv"):
            os.remove(f"../games/{filename}")

Die Funktion `create_player` bekommt eine Spielerklasse, einen Spielernamen sowie die Konfiguration für alle Spieler und gibt eine Instanz der Spielerklasse zurück.

In [ ]:
def create_player(player: type, name: str, config: dict) -> ChessAI:
    """Instantiates a player with the given config"""
    return player(player_name=name, **config[player.__name__])

Die Funktion `is_same_config` bekommt eine ChessAI-Klasse die sowohl Spieler, als auch Gegner ist und prüft ob die Konfiguration der beiden Positionen identisch ist. Diese Methode wird genutzt um doppelte Spielerpaare mit gleicher Konfiguration zu vermeiden.

In [ ]:
def is_same_config(player: type, config) -> bool:
    return config["players"][player.__name__] == config["opponents"][player.__name__]

Die Methode `build_pairs` bekommt zwei Listen von ChessAI-Klassen, eine Konfiguration als Dictionary. Sie bildet das kartesische Produkt der beiden Listen, wobei die jeweiligen Klassen dabei instanziiert werden mit der gegebenen Konfiguration.
Optional können dabei reflexive Paare (Spieler = Gegner) vermieden werden und symmetrische Paare (Farbenwechsel) erzeugt werden.

In [ ]:
def build_pairs(players: list[type], opponents: list[type], config: dict) -> set[tuple[ChessAI, ChessAI]]:
    """Builds cartesian pairs of two iterables"""
    make_symmetric_pairs = config["make_symmetric_pairs"]
    exclude_reflexive_pairs = config["exclude_reflexive_pairs"]
    result: set[tuple[ChessAI, ChessAI]] = set()
    for player in players:
        for opponent in opponents:
            if exclude_reflexive_pairs and player == opponent:
                continue
            player_1 = create_player(player, "White", config["players"])
            player_2 = create_player(opponent, "Black", config["opponents"])
            result.add((player_1, player_2))
            if make_symmetric_pairs and (not player in opponents or not is_same_config(player, config)):
                s_player_1 = create_player(opponent, "White", config["opponents"])
                s_player_2 = create_player(player, "Black", config["players"])
                result.add((s_player_1, s_player_2))
    return result

Die `build_player_pairs`-Funktion ermittelt die angegebenen Spieler und Gegner und bildet eine Liste von Spielerpaaren und gibt diese nach dem Klassennamen sortiert zurück.


In [ ]:
def build_player_pairs(config: dict) -> list[ChessAI]:
    """Builds the sorted cartesian product of players x opponents and opponents x players"""
    # Get players and opponents
    players   = [player for player in available_versions 
                    if player.__name__ in config["players"]]
    opponents = [player for player in available_versions 
                    if player.__name__ in config["opponents"]]
    # Build and return pairs
    player_pairs = build_pairs(players, opponents, config)
    return sorted(player_pairs, key=lambda x: (type(x[0]).__name__, type(x[1]).__name__))

Die `run`-Funktion nimmt eine Liste von Spielerpaaren und führt für jedes Paar die gewünschte Anzahl an Spielen mit dem gewünschten Seed durch.

In [ ]:
def run(player_pairs: list[tuple[ChessAI, ChessAI]], config: dict) -> None:
    """Runs one or more games with each player pair."""
    seed = config["seed"]
    repetitions = config["repetitions"]
    # Let all player pairs play
    for i, players in enumerate(player_pairs):
        white_player, black_player = players
        print(f"Playing game {i+1} of {len(player_pairs)}")
        run_games(white_player, black_player, repetitions, seed)

Die `collect_results`-Funktion importiert die Ergebnisse aus allen `.pgn`-Dateien im `games`-Ordner und bereitet sie als Dictionary auf.

In [ ]:
# Collect results from .pgn files
import chess.pgn

def collect_results() -> dict[str, list]:
    """Scans all .pgn files inside the games folder and collects the game results as dictionary."""
    # All results are presented from the view of the white color
    results_map_white = {"1-0": "Won", "0-1": "Lost", "1/2-1/2": "Remis", "*": "Remis"}
    results_map_black = {"0-1": "Won", "1-0": "Lost", "1/2-1/2": "Remis", "*": "Remis"}
    games_map: dict[str, list] = {}

    # Iterate over all .pgn files
    for filename in os.listdir("../games"):
        if not filename.endswith(".pgn"):
            continue
        with open(os.path.join("..", "games", filename)) as pgn:
            pgn_game = chess.pgn.read_game(pgn)
        if not pgn_game:
            continue

        # Collect results
        results = {
            "round": pgn_game.headers["Round"],
            "opponent": pgn_game.headers["Black"],
            "result_white": results_map_white[pgn_game.headers["Result"]],
            "result_black": results_map_black[pgn_game.headers["Result"]],
            "outcome": pgn_game.headers["Outcome"],
            "duration": pgn_game.headers["Duration"],
            "seed": pgn_game.headers["Seed"],
            "depth_white": pgn_game.headers["Depth"].split("-")[0],
            "depth_black": pgn_game.headers["Depth"].split("-")[1],
            "total_cache_hits_white": pgn_game.headers["CacheHits"].split("-")[0],
            "total_cache_hits_black": pgn_game.headers["CacheHits"].split("-")[1],
            "state_changes": {node.comment: node.ply() for node in pgn_game.mainline() if node.comment},
            "total_moves": pgn_game.end().ply(),
            "filename": filename,
        }

        # Save results (uses white player name as key)
        entry = games_map.get(pgn_game.headers["White"], [])
        entry.append(results)
        games_map[pgn_game.headers["White"]] = entry
    return games_map

Die `export_to_csv`-Funktion exportiert eine gegebene Sammlung (Dictionary) von Spielergebnissen als CSV-Datei.

In [ ]:
import csv
from datetime import datetime

def export_to_csv(games_map: dict[str, list]) -> None:
    """Exports the given results dictionary to csv."""
    filename = f'statistics_{datetime.now().strftime("%Y-%m-%d_%H-%M-%S-%f")}.csv'
    with open(os.path.join("..", "games", filename), 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        # Write headlines
        csv_writer.writerow([
            "Player", 
            "Result white",
            "Depth white",
            "Cache hits white",
            "Opponent", 
            "Result black",
            "Depth black",
            "Cache hits black",
            "Outcome",
            "Moves", 
            "Duration", 
            "Moves to middle game", 
            "Moves to end game", 
            "Round", 
            "Seed",
            "File",
        ])
        # Write results
        for player in sorted(games_map):
            games = sorted(games_map[player], key=lambda game: game["filename"])
            for game in games:
                csv_writer.writerow([
                    player, 
                    game["result_white"],
                    game["depth_white"],
                    game["total_cache_hits_white"],
                    game["opponent"], 
                    game["result_black"],
                    game["depth_black"],
                    game["total_cache_hits_black"],
                    game["outcome"].split(".")[1],
                    game["total_moves"], 
                    game["duration"].split(".")[0], 
                    game["state_changes"].get("State.MIDDLE_GAME", "-"), 
                    game["state_changes"].get("State.END_GAME", "-"), 
                    game["round"].replace("/", " of "),
                    game["seed"],
                    game["filename"],
                ])

Die folgende Zelle bereinigt den `games`-Ordner, erstellt die Spielerpaare, startet die Spiele, sammelt die Ergebnisse und exportiert sie als CSV-Datei.

In [ ]:
%%time
# Run and export games
#cleanup_game_folder()
config = get_config()
player_pairs = build_player_pairs(config)
#run(player_pairs, config)
#results = collect_results()
#export_to_csv(results)
player_pairs